# Spotify Tracks analysis

Análise exploratória de uma amostra dos dados da biblioteca do Spotify no inverno de 2019. A análise foi realizada utilizando-se o framework Apache Spark e a biblioteca para a ciência de dados Pandas, em menor medida.

A análise exploratória consiste nas seguintes etapas:

1. Leitura remota da base de dados e exibição do schema inicial do data frame;
2. Remoção de colunas irrelevantes para a análise de dados;
3. Adição de duas novas colunas no data frame;
4. Análise dos dados das colunas recém adicionadas;
5. Análise da média das popularidades mais altas dos tracks agrupados por artista;
6. Análise da média da popularidade mais baixa dos tracks agrupados por artista;
7. Análise da popularidade dos tracks explícitos agrupados por artista;
8. Considerações finais;

A base de dados pode ser obtida gratuitamente [aqui](https://raw.githubusercontent.com/ThinkR-open/datasets/master/tracks.csv).


## 1. Leitura remota da base de dados e exibição do schema inicial do data frame

A base de dados está armazenada remotamente em um repositório do GitHub. Para a leitura dos dados utiliza-se a biblioteca Pandas, visto que o PySpark faz a leitura apenas de arquivos locais.

Após a leitura do data frame com o Pandas, cria-se um novo data frame com o PySpark.


In [ ]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql import SparkSession


spark = (SparkSession
         .builder
         .appName("Spotify Tracks Analysis")
         .getOrCreate()
         )


In [ ]:
pdf = pd.read_csv(
    'https://raw.githubusercontent.com/ThinkR-open/datasets/master/tracks.csv')
df = spark.createDataFrame(pdf)

df.printSchema()


## 2. Remoção de colunas irrelevantes para a análise de dados

Como visto, algumas colunas no data frame são irrelevantes para a anállise exploratória dos dados. Portanto, é necessário fazer a remoção destas colunas, consequentemente tornando o data frame mais compacto.


In [ ]:
df = df.drop("trackid", "track_uri", "album_id", "type", "uri",
             "track_href", "analysis_url", "time_signature")

df.printSchema()


## 3. Adição de duas novas colunas no data frame

Duas novas colunas são adicionadas no data frame, **duration_minutes** e **duration_seconds**.

A coluna **duration_seconds** é criada com base na coluna **duration_ms**, dividindo esta por 1000 e depois usando a função **cast** para transformar a coluna para o tipo inteiro (INT).

Por sua vez, a coluna **duration_minutes** é criada com base na coluna **duration_seconds**, dividindo esta por 60 e depois usando a função **cast** para transformar a coluna para o tipo inteiro (INT).


In [ ]:
df = df.withColumn("duration_seconds", (df["duration_ms"]/1000).cast("INT"))
df = df.withColumn("duration_minutes", (df["duration_seconds"]/60).cast("INT"))


## 4. Análise dos dados das colunas recém adicionadas

Exibe uma descrição estatística das colunas **duration_minutes** e **duration_seconds**.

Como visto, todos os tracks tem uma média de 3,52 minutos. O track com a menor duração tem 5 segundos, e o track com a maior duração tem 21 minutos.


In [ ]:
(df
 .select("duration_minutes", "duration_seconds")
 .summary()
 .show()
 )


## 5. Análise da média das popularidades mais altas dos tracks agrupados por artista

Primeiro agrupa-se o data fframe por artista. Depois, calcula a média da popularidade de cada track e a quantidade de tracks para cada um dos artistas. Ordena o data frame de forma descendente, e limita o dataframe para 10 linhas.


In [ ]:
high_popularity = (df
                   .groupBy("artist")
                   .agg({"popularity": "mean", "track": "count"})
                   .orderBy(["avg(popularity)", "count(track)"], ascending=[False, False])
                   .limit(10)
                   )

high_popularity.show()


## 6. Análise da média da popularidade mais baixa dos tracks agrupados por artista

Primeiro agrupa-se o data fframe por artista. Depois, calcula a média da popularidade de cada track e a quantidade de tracks para cada um dos artistas. Ordena o data frame de forma ascendente, e limita o dataframe para 10 linhas.


In [ ]:
low_popularity = (df
                  .groupBy("artist")
                  .agg({"popularity": "mean", "track": "count"})
                  .orderBy(["avg(popularity)", "count(track)"], ascending=[True, False])
                  .limit(10)
                  )

low_popularity.show()


## 7. Análise da popularidade dos tracks explícitos agrupados por artista

Filtra o data frame apenas por tracks explícitos. Depois, agrupa o data frame por artista, e calcula a média da popularidade de cada track junto com a quantidade de cada track por artista.


In [12]:
explicit_tracks = (df[df["explicit"]]
                   .groupBy("artist")
                   .agg({"track": "count", "popularity": "mean"})
                   .orderBy("count(track)", ascending=False)
                   .limit(10)
                   .select(F.col("artist"), F.col("count(track)").alias("explicit_tracks"), F.col("avg(popularity)").alias("avg_popularity"))
                   )

explicit_tracks.show()


+--------------------+---------------+------------------+
|              artist|explicit_tracks|    avg_popularity|
+--------------------+---------------+------------------+
|             Pantera|             28| 39.17857142857143|
|              Eminem|             21| 6.571428571428571|
|           Benighted|             18|               0.0|
|           Bloodbath|             14|29.857142857142858|
|Bring Me The Horizon|             14| 38.07142857142857|
|      Marilyn Manson|             13|1.2307692307692308|
|           Deez Nuts|             11|21.636363636363637|
|Here Comes The Kr...|             10|              15.4|
|              Psykup|             10|               9.1|
|      Richard Cheese|              9|32.666666666666664|
+--------------------+---------------+------------------+



In [ ]:
(explicit_tracks
 .select("explicit_tracks", "avg_popularity")
 .summary()
 .show()
 )


## Considerações finais

Como analisado, concluiu-se que:

- A duração média dos tracks foi de 3,52 minutos, com a maior duração sendo de 21 minutos e a menor de 5 segundos;
- O artista com a maior média de popularidade de seus tracks foi **Eurythmics**, com 81 pontos de popularidade e 1 track;
- O artista com a menor média de popularidade de seus tracks foi **Kreator**, com 0 pontos e 63 tracks;
- O artista com mais tracks explícitos foi **Pantera**, com uma média de 39,18 pontos de popularidade com 28 tracks;
- Cada artista com tracks explícitos teve em média 14 tracks, com uma média de popularidade de 19,37 pontos;
- A média mais alta de popularidade foi 39,17 pontos, e a média de tracks explícitos por artista foi de 28 tracks;
